# task
Участие в соревновании https://github.com/dialogue-evaluation/RuREBus по NER и RE.
Форма сдачи: ноутбук с кодом и метриками качества ваших моделей.
Оцениваение: минимально в 2 раза больше обычного дз (если сделаете совсем много, возможны дополнительные бонусы).
На чуть больше половины баллов за проект достаточно правильно сделать CharCNN-BLSTM-CRF для NER и вариант модели Miwa & Bansal (2016) для RE.
Дедлайн 1 мая 9 утра (4 недели)

описание разметки текста: https://github.com/dialogue-evaluation/RuREBus/blob/master/markup_instruction.pdf

# libs

In [1]:
%%writefile requirements.txt
razdel
pytorch-crf

Writing requirements.txt


In [2]:
!pip install --upgrade -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuREBus/master/eval_scripts/brat_format.py
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuREBus/master/eval_scripts/evaluate_ners.py

--2023-05-02 11:56:40--  https://raw.githubusercontent.com/dialogue-evaluation/RuREBus/master/eval_scripts/brat_format.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5030 (4.9K) [text/plain]
Saving to: ‘brat_format.py’

brat_format.py      100%[===================>]   4.91K  --.-KB/s    in 0s      

2023-05-02 11:56:40 (68.6 MB/s) - ‘brat_format.py’ saved [5030/5030]

--2023-05-02 11:56:40--  https://raw.githubusercontent.com/dialogue-evaluation/RuREBus/master/eval_scripts/evaluate_ners.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 

In [4]:
from glob import glob
from razdel import sentenize, tokenize
from collections import namedtuple
from itertools import chain
import random
import numpy as np
import time
from tqdm import tqdm

import torch
from torch import nn
import torch.optim as optim

from torchcrf import CRF #https://pytorch-crf.readthedocs.io/en/stable/

from evaluate_ners import compute_precision_and_recall

# 1. dataset

## 1.1 download

In [5]:
#удаление файлов
for i in glob('*'):
  !rm -rf {i}

# # чек типа файла
# !apt install file
# !file RuREBus.git

#создание папки
!mkdir train_data
!mkdir test_data

In [6]:
!git clone https://github.com/dialogue-evaluation/RuREBus.git

Cloning into 'RuREBus'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 247 (delta 16), reused 4 (delta 1), pack-reused 217
Receiving objects: 100% (247/247), 14.22 MiB | 6.72 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [7]:
for i in glob('RuREBus/train_data/*'):
  !unzip "{i}" -d "train_data"

for i in glob('RuREBus/test_data/test_full*'):
  !unzip "{i}" -d "test_data"

from IPython.display import clear_output
clear_output()

glob('train_data/*')

['train_data/train_part_2',
 'train_data/train_part_1',
 'train_data/train_part_3']

## 1.2 preprocessing

### 1.2.1 sentenize & tokenize

In [8]:
def get_tokenized_dataset(path_to_files: str):
    """
    take txt files and tokenize them by razdel liba 

    params: path_to_files - path to train files
    return: dict[path] = (file_sentences, file_tokenize)
            file_sentences - [ Substring(start, stop, sentence),...]
            file_tokenize  - [ Substring(start, stop, token),...]
    """
    result = {}
    for file_i in glob(path_to_files+'/*.txt'):
      short_file_i = file_i.replace(path_to_files+'/', '').replace('.txt','')
      file_txt = open(file_i, 'r', encoding='utf-8').read()
      file_sentences = list(sentenize(file_txt))
      file_tokenize = [[tok for tok in tokenize(sent.text)] for sent in file_sentences ]
      result[short_file_i] = (file_sentences, file_tokenize)
    return result

In [9]:
a = get_tokenized_dataset("train_data/train_part_1")
len(a['20336241021100524345002_22_part_1'][0])

13

### 1.2.2 сущности (NER) и отношения (RE)

In [10]:
entuple = namedtuple('entuple', ['name', 'start', 'stop', 'text'])
retuple = namedtuple('retuple', ['name', 'arg1', 'arg2'])

def get_annotated_dataset(path_to_files: str):
    """
    take ann files and parse them

    params: path_to_files - path to train files
    return: dict[path] = {'entity': entities_dict,
                          'relate': relations_dict}
            entities_dict  - {entity_id: ['name', 'start', 'stop', 'text']}
            relations_dict - {relate_id: ['name', 'arg1', 'arg2']}
                             arg1, arg2 taken from list of entity_id
    """
    result = {}
    for file_i in glob(path_to_files+'/*.ann'):
        short_file_i = file_i.replace(path_to_files+'/', '').replace('.ann','')

        file_txt_list = open(file_i, 'r', encoding='utf-8').readlines()
        file_txt_list = [i.strip().split('\t') for i in file_txt_list]

        if len(file_txt_list) > 0:
            # сущности
            entities = [i for i in file_txt_list if i[0][0]=='T']
            entities = [(i[0], i[1], ' '.join(i[2:])) if len(i)>3 else i for i in entities] #на случай если в тексте была табуляция
            assert len(entities) == len(set([i[0] for i in entities])), 'allert ent #1: имена сущностей (T***) не уникальны'
            assert min([len(i[1].split()) for i in entities]) == max([len(i[1].split()) for i in entities]), 'allert ent #2: что-то не так в определении сущности'

            entities_dict = dict( [(tag, entuple(attr.split()[0], int(attr.split()[1]), int(attr.split()[2]), text))
                                      for (tag, attr, text) in entities] )
            # отношения
            relations = [i for i in file_txt_list if i[0][0]=='R']
            assert len(relations) == len(set([i[0] for i in relations])), 'allert rel #1: имена сущностей (R***) не уникальны'
            relations_dict = dict( [(tag, retuple(attr.split()[0], attr.split()[1].split(':')[1], attr.split()[2].split(':')[1]))
                                      for (tag, attr) in relations] )
            assert len( set( [i.arg1 for i in relations_dict.values()] + [i.arg2 for i in relations_dict.values()] ) - set(entities_dict.keys()) ) == 0, "allert rel #2: встречены сущности не встречающиеся в entities"

            result[short_file_i] = {'entity': entities_dict,
                                    'relate': relations_dict}
    return result

In [11]:
b = get_annotated_dataset('train_data/train_part_1')
len(b['31339011021101006981005_6_part_0']['entity'])

132

### 1.2.3 сущности текущего текста

In [12]:
def get_entities_from_text(sent_start, sent_stop, ent_dict):
    """
    filtering entities_dict taking information about the boundaries of the sentence

    params: sent_start - start of sentence
            sent_stop  - end of sentence
            ent_dict - dictionary of entities
    return: entities_dict = {entity_id: ['name', 'start', 'stop', 'text']}
    """
    entity_filter_list = [ entity_id for entity_id, (_, entity_start, entity_stop, _) in ent_dict['entity'].items()
                      if sent_start <= entity_start <= entity_stop <= sent_stop ]
    result = {'entity': {entity_id: ent_dict['entity'][entity_id] for entity_id in entity_filter_list}}
    return result

### 1.2.4 словари сущностей и отношений

In [13]:
entity_set = set()
relate_set = set()
for path_to_files in ['train_data/train_part_1', 'train_data/train_part_2', 'train_data/train_part_3']:
    annotated_dataset_i = get_annotated_dataset(path_to_files)
    for file_i in annotated_dataset_i:
      for _, (ent_name,_,_,_) in annotated_dataset_i[file_i]['entity'].items():
          entity_set = entity_set.union(set([ent_name]))
      for _, (rel_name,_,_) in annotated_dataset_i[file_i]['relate'].items():
          relate_set = relate_set.union(set([rel_name]))

entity_list = [['B-'+x, 'I-'+x] for x in entity_set]
entity_list = list(chain(*entity_list))

entity_dict = {entity_list[i]: i+1 for i in range(len(entity_list))}
entity_dict['O'] = 0

rel_2_id = {y:x for x,y in enumerate(relate_set)}
id_2_rel = {x:y for x,y in enumerate(relate_set)}

print('entity dictionary:')
print(entity_dict)

print('\nrelate dictionary:')
print(rel_2_id)

entity dictionary:
{'B-CMP': 1, 'I-CMP': 2, 'B-INST': 3, 'I-INST': 4, 'B-QUA': 5, 'I-QUA': 6, 'B-SOC': 7, 'I-SOC': 8, 'B-ACT': 9, 'I-ACT': 10, 'B-BIN': 11, 'I-BIN': 12, 'B-ECO': 13, 'I-ECO': 14, 'B-MET': 15, 'I-MET': 16, 'O': 0}

relate dictionary:
{'GOL': 0, 'PNG': 1, 'FNG': 2, 'TSK': 3, 'PNT': 4, 'NNT': 5, 'FNT': 6, 'NPS': 7, 'NNG': 8, 'PPS': 9, 'FPS': 10}


In [14]:
# # в качестве доп инфо для RE
# ner_dict = {y:x for x,y in enumerate(entity_set)}
# ner_dict

# 2. NER

## 2.1 BIO разметка
B - begin, I - inner, O - outer

In [ ]:
Sample = namedtuple("Sample", "text, tokens, labels")

def get_ner_dataset(path_file_list: list):
    samples = []
    for path_to_files in path_file_list:

        # path_to_files = 'train_data/train_part_1'

        tokenized_dataset = get_tokenized_dataset(path_to_files)
        annotated_dataset = get_annotated_dataset(path_to_files)

        for file_name in tokenized_dataset:
            #print(file_name)
            sentences, tokenized_sentences = tokenized_dataset[file_name] #[ Substring(start, stop, sentence),...]
            try:
                annotations = annotated_dataset[file_name]                    #[ Substring(start, stop, token),...]
            except KeyError:
                continue

            for sent_id, (sentence, tokenized_sentence) in enumerate(zip(sentences, tokenized_sentences)):
                entities_from_sentence = get_entities_from_text(sentence.start, sentence.stop, annotations) #{entity_id: ['name', 'start', 'stop', 'text']}
                tokens = [token.text for token in tokenized_sentence]

                if len(entities_from_sentence['entity'])==0:
                    labels = [entity_dict['O']]*len(tokenized_sentence)
                    # labels = [entity_dict[i] for i in labels]
                else:
                    labels = []
                    for token in tokenized_sentence:
                      label = entity_dict['O']
                      for _, (ent_name,ent_start,ent_end,_) in entities_from_sentence['entity'].items(): #{entity_id: ['name', 'start', 'stop', 'text']}
                          if sentence.start+token.start == ent_start:
                              label = 'B-'+ent_name
                              label = entity_dict.get(label, 'O')
                          elif sentence.start+token.start > ent_start and sentence.start+token.stop <= ent_end:
                              label = 'I-'+ent_name
                              label = entity_dict.get(label, 'O')
                      labels.append(label)

                sample = Sample(sentence.text, tokens, labels)
                samples.append(sample)

    return samples

## 2.2 train-test split

In [ ]:
data = get_ner_dataset(glob('train_data/*'))
random.shuffle(data)

train_size = int(len(data)*0.8)
train = data[:train_size]
val = data[train_size:]
test = get_ner_dataset(['test_data'])

множество слов и символов

In [ ]:
char_set = list( {ch for sample in train for token in sample.tokens for ch in token} )
char_set.insert(0, '<unk>'), char_set.insert(0, '<pad>')
word_set = list( {token for sample in train for token in sample.tokens} )
word_set.insert(0, '<unk>'), word_set.insert(0, '<pad>')

(None, None)

## 2.3 генерация батчей

In [ ]:
# max_seq_len=500
# max_char_seq_len=40
# batch_size=32
def get_next_gen_batch(samples, max_seq_len=512, max_char_seq_len=40, batch_size=2):
    indices = np.arange(len(train))
    np.random.shuffle(indices)
    batch_begin = 0
    with tqdm(total=len(train)) as pbar:
      while batch_begin < len(train):
          batch_indices = indices[batch_begin: batch_begin + batch_size]
          batch_words = []
          batch_chars = []
          batch_labels = []
          batch_max_len = 0
          batch_masks = [] #for CRF
          for data_ind in batch_indices:
              sample = train[data_ind] #беру одно предложение
              words = torch.zeros(max_seq_len, dtype=torch.long).cuda()
              inputs = torch.zeros((max_seq_len, max_char_seq_len), dtype=torch.long).cuda()
              for token_num, token in enumerate(sample.tokens[:max_seq_len]): #цикл по токенам предложения, обрезанного до max_seq_len
                  #слова
                  words[token_num] = word_set.index(token) if token in word_set else word_set.index('<unk>')
                  #символы
                  for char_num, char in enumerate(token[:max_char_seq_len]):
                      inputs[token_num][char_num] = char_set.index(char) if char in char_set else char_set.index('<unk>')
              labels = sample.labels[:max_seq_len]         #аналогично с labels
              masks = [1]*len(labels) + [0]*(max_seq_len - len(labels)) 
              labels += [0] * (max_seq_len - len(labels))  #аналогично с labels
              
              batch_words.append(words)
              batch_chars.append(inputs)
              batch_labels.append(labels)
              batch_masks.append(masks)
              
          batch_begin += batch_size
          pbar.update(batch_size)
          
          batch_words = torch.stack(batch_words)
          batch_chars = torch.stack(batch_chars)
          labels = torch.cuda.LongTensor(batch_labels)
          batch_masks = torch.tensor(batch_masks).cuda()>0
          yield batch_indices, batch_words, batch_chars, labels, batch_masks

In [ ]:
for _, batch_words, batch_chars, labels, _ in get_next_gen_batch(train):
    1+1
    break
print()
print( batch_words.size() )
print( batch_chars.size() )
print( labels.size() )

  0%|          | 2/8368 [00:00<21:48,  6.39it/s]


torch.Size([2, 512])
torch.Size([2, 512, 40])
torch.Size([2, 512])


## 2.4 модели

In [ ]:
%%html
<img src='https://user-images.githubusercontent.com/31881382/35037901-f4a69b14-fb89-11e7-8c28-08a0cb1310cb.png' />

In [ ]:
class SuperSimpleModel(nn.Module):
    def __init__(self, char_set_size, char_embedding_dim=16, classes_count=len(entity_dict), char_max_seq_len=40):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.out_layer = nn.Linear(char_max_seq_len * char_embedding_dim, classes_count)

    def forward(self, words, chars):
        projections = self.embeddings_layer(chars)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        output = self.out_layer(projections)
        return output

class CharCNNBiLSTM(nn.Module):
    def __init__(self, char_set_size, word_set_size,
                 char_embedding_dim=8, word_embedding_dim=64,
                 classes_count=len(entity_dict), 
                 char_max_seq_len=40, lstm_embedding_dim=16,
                 kernel_size=3, max_seq_len=512 ):
        super().__init__()
        self.char_max_seq_len = char_max_seq_len
        self.char_embedding_dim = char_embedding_dim
        self.max_seq_len = max_seq_len

        self.char_embedding = nn.Embedding(char_set_size, char_embedding_dim)
        self.char_cnn = nn.Conv1d(in_channels=char_embedding_dim, out_channels=1, kernel_size=kernel_size)
        self.word_embedding = nn.Embedding(word_set_size, word_embedding_dim  )
        self.dropout = nn.Dropout(0.2)
        self.lstm_layer = nn.LSTM((char_max_seq_len-kernel_size+1)+word_embedding_dim, lstm_embedding_dim, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim*2, classes_count)

    def forward(self, words, chars):
        #работа с символами
        c_emb = self.char_embedding(chars)
        c_emb = c_emb.reshape(c_emb.size(0)*c_emb.size(1), self.char_max_seq_len, self.char_embedding_dim)
        c_emb = c_emb.permute(0,2,1)

        c_cnn = self.char_cnn(c_emb)
        c_cnn = c_cnn.reshape(chars.size(0), self.max_seq_len, -1)

        #работа со словами
        w_emb = self.word_embedding(words)
        com_emb = torch.cat((c_cnn, w_emb), 2)
        rnn_output, _ = self.lstm_layer(com_emb)
        result = self.out_layer.forward(rnn_output)

        return result

один проход - отладка

In [ ]:
model = CharCNNBiLSTM(len(char_set), len(word_set))

lr=0.01
device_name="cuda"
device = torch.device(device_name)

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss().cuda()
model = model.to(device)

logits = model(batch_words, batch_chars) # Прямой проход
logits = logits.transpose(1, 2) # crossentropy require (N,C,d,d,d,d,d), N-batch size, C-number of classes
loss = loss_function(logits, labels) # Подсчёт ошибки
loss.backward() # Подсчёт градиентов dL/dw
optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации



## 2.5 ф-ция обучения

In [ ]:
def train_gen_model(model, train_samples, val_samples, epochs_count=10, 
                    loss_every_nsteps=100, lr=0.01, save_path="model.pt", device_name="cuda",
                    early_stopping=True):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss().cuda()
    prev_avg_val_loss = None
    for epoch in range(epochs_count):
        model.train()
        for step, (_, batch_words, batch_chars, batch_labels, _) in enumerate(get_next_gen_batch(train)):
            logits = model(batch_words, batch_chars) # Прямой проход
            logits = logits.transpose(1, 2)
            loss = loss_function(logits, batch_labels) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
        val_total_loss = 0
        val_batch_count = 0
        model.eval()
        for _, (_, batch_words, batch_chars, batch_labels, _) in enumerate(get_next_gen_batch(val)):
            logits = model(batch_words, batch_chars) # Прямой проход
            logits = logits.transpose(1, 2)
            val_total_loss += loss_function(logits, batch_labels) # Подсчёт ошибки
            val_batch_count += 1
        avg_val_loss = val_total_loss/val_batch_count
        print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
        total_loss = 0
        start_time = time.time()

        if early_stopping and prev_avg_val_loss is not None and avg_val_loss > prev_avg_val_loss:
            model.load_state_dict(torch.load(save_path))
            model.eval()
            break
        prev_avg_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)

## 2.6 метрики

In [ ]:
def predict(model, samples):
    model.eval()
    tp, fp, fn = 0,0,0
    for _, (indices, batch_words, batch_chars, batch_labels, _) in enumerate(get_next_gen_batch(samples)):
        logits = model(batch_words, batch_chars)
        plabels = logits.max(dim=2)[1]
        tp += len([(i,j) for i,j in zip(batch_labels[0].tolist(), plabels[0].tolist()) if (j!=0) and (i==j)])
        fp += len([(i,j) for i,j in zip(batch_labels[0].tolist(), plabels[0].tolist()) if (j!=0) and (i!=j)])
        fn += len([(i,j) for i,j in zip(batch_labels[0].tolist(), plabels[0].tolist()) if (j==0) and (i!=j)])

    recall, precision = compute_precision_and_recall(tp, fp, fn)
    try:
      f_measure = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
      f_measure = 'None'
    print('precision:', precision)
    print('recall:   ', recall)
    print('f1:       ', f_measure)

## --SimpleModel

In [ ]:
model = SuperSimpleModel(len(char_set))
train_gen_model(model, train, val, epochs_count=1, early_stopping=False, lr=0.02)

Trainable params: 13681
Epoch = 0, Avg Train Loss = 0.4852, Avg val loss = 0.0956, Time = 460.83s


In [ ]:
print('on train dataset:'.center(30, '-'))
predict(model, train)

print('\n', 'on val dataset:'.center(30, '-'))
predict(model, val)

print('\n', 'on test dataset:'.center(30, '-'))
predict(model, test)

------on train dataset:-------
precision: 0.38149556400506973
recall:    0.09480314960629921
f1:        0.1518668012108981

 -------on val dataset:--------
precision: 0.3604060913705584
recall:    0.0849536344600658
f1:        0.13749697409828127

 ------on train dataset:-------
precision: 0.3703271028037383
recall:    0.09559710494571773
f1:        0.15196548418024927


## --CharCNN + BiLSTM

In [ ]:
model = CharCNNBiLSTM(len(char_set), len(word_set))
train_gen_model(model, train, val, epochs_count=1, early_stopping=False, lr=0.02)

Trainable params: 1537706
Epoch = 0, Avg Train Loss = 0.2857, Avg val loss = 0.0759, Time = 488.59s


In [ ]:
print('on train dataset:'.center(30, '-'))
predict(model, train)

print('\n', 'on val dataset:'.center(30, '-'))
predict(model, val)

print('\n', 'on test dataset:'.center(30, '-'))
predict(model, test)

------on train dataset:-------
precision: 0.3448667044809983
recall:    0.26331745344304897
f1:        0.2986247544204323

 -------on val dataset:--------
precision: 0.31992687385740404
recall:    0.21739130434782608
f1:        0.25887573964497046

 ------on train dataset:-------
precision: 0.336644591611479
recall:    0.25459098497495825
f1:        0.2899239543726236


In [ ]:
# model.eval()
# for step, (_, batch_words, batch_chars, batch_labels, _) in enumerate(get_next_gen_batch(train)):
#             # print(batch)
#             logits = model(batch_words, batch_chars)
#             plabels = logits.max(dim=1)[1]
#             # print(logits)
#             # print(plabels)
#             # print()
#             # break

## --CharCNN + BiLSTM + CRF

#### без использования масок паддинга

In [ ]:
def train_gen_model(model, train_samples, val_samples, epochs_count=10, 
                    loss_every_nsteps=100, lr=0.01, save_path="model.pt", device_name="cuda",
                    early_stopping=True):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = CRF(len(entity_dict), batch_first=True).cuda()
    prev_avg_val_loss = None
    for epoch in range(epochs_count):
        model.train()
        for step, (_, batch_words, batch_chars, batch_labels, _) in enumerate(get_next_gen_batch(train)):
            logits = model(batch_words, batch_chars) # Прямой проход
            loss = -loss_function(logits, batch_labels) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
            print(loss.item())
        val_total_loss = 0
        val_batch_count = 0
        model.eval()
        for _, (_, batch_words, batch_chars, batch_labels, _) in enumerate(get_next_gen_batch(val)):
            logits = model(batch_words, batch_chars) # Прямой проход
            val_total_loss += -loss_function(logits, batch_labels) # Подсчёт ошибки
            val_batch_count += 1
        avg_val_loss = val_total_loss/val_batch_count
        print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
        total_loss = 0
        start_time = time.time()

        if early_stopping and prev_avg_val_loss is not None and avg_val_loss > prev_avg_val_loss:
            model.load_state_dict(torch.load(save_path))
            model.eval()
            break
        prev_avg_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)

In [ ]:
model = CharCNNBiLSTM(len(char_set), len(word_set))
train_gen_model(model, train, val, epochs_count=1, early_stopping=False, lr=0.02)

Trainable params: 1574290


/usr/local/lib/python3.10/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:493.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch = 0, Avg Train Loss = 8280.9223, Avg val loss = 1504.9847, Time = 576.71s


In [ ]:
print('on train dataset:'.center(30, '-'))
predict(model, train)

print('\n', 'on val dataset:'.center(30, '-'))
predict(model, val)

print('\n', 'on test dataset:'.center(30, '-'))
predict(model, test)

------on train dataset:-------
precision: 0.26618705035971224
recall:    0.012701682114658427
f1:        0.024246395806028834

 -------on val dataset:--------
precision: 0.22758620689655173
recall:    0.01071776550828191
f1:        0.020471464019851116

 ------on train dataset:-------
precision: 0.23076923076923078
recall:    0.010361067503924647
f1:        0.019831730769230768


#### с использованием масок паддинга

In [ ]:
def train_gen_model(model, train_samples, val_samples, epochs_count=10, 
                    loss_every_nsteps=100, lr=0.01, save_path="model.pt", device_name="cuda",
                    early_stopping=True):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = CRF(len(entity_dict), batch_first=True).cuda()
    prev_avg_val_loss = None
    for epoch in range(epochs_count):
        model.train()
        for step, (_, batch_words, batch_chars, batch_labels, batch_masks) in enumerate(get_next_gen_batch(train)):
            logits = model(batch_words, batch_chars) # Прямой проход
            loss = -loss_function(logits, batch_labels, batch_masks) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
        val_total_loss = 0
        val_batch_count = 0
        model.eval()
        for _, (_, batch_words, batch_chars, batch_labels, batch_masks) in enumerate(get_next_gen_batch(val)):
            logits = model(batch_words, batch_chars) # Прямой проход
            val_total_loss += -loss_function(logits, batch_labels, batch_masks) # Подсчёт ошибки
            val_batch_count += 1
        avg_val_loss = val_total_loss/val_batch_count
        print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
        total_loss = 0
        start_time = time.time()

        if early_stopping and prev_avg_val_loss is not None and avg_val_loss > prev_avg_val_loss:
            model.load_state_dict(torch.load(save_path))
            model.eval()
            break
        prev_avg_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)

In [ ]:
model = CharCNNBiLSTM(len(char_set), len(word_set))
train_gen_model(model, train, val, epochs_count=1, early_stopping=False, lr=0.02)

In [ ]:
print('on train dataset:'.center(30, '-'))
predict(model, train)

print('\n', 'on val dataset:'.center(30, '-'))
predict(model, val)

print('\n', 'on test dataset:'.center(30, '-'))
predict(model, test)

# 3. RE with NEs

постановка задачи: на основе текста и выделенных в нем сущностей спрогнозировать связь между ними

## 3.1 разметка

In [15]:
Sample = namedtuple("Sample", "text, tokenized_text, tokens, e1, e2, label")

def get_re_dataset(path_file_list: list):
    samples = []
    for path_to_files in path_file_list:
        # path_to_files = 'train_data/train_part_1'

        tokenized_dataset = get_tokenized_dataset(path_to_files)
        annotated_dataset = get_annotated_dataset(path_to_files)

        for file_name in tokenized_dataset:
            #print(file_name)
            sentences, tokenized_sentences = tokenized_dataset[file_name] #[ Substring(start, stop, sentence),...]
            try:
                annotations = annotated_dataset[file_name]                #[ Substring(start, stop, token),...]
            except KeyError:
                annotations = None

            if annotations:
                for sent_id, (sentence, tokenized_sentence) in enumerate(zip(sentences, tokenized_sentences)):
                    entities_from_sentence = get_entities_from_text(sentence.start, sentence.stop, annotations) #{entity_id: ['name', 'start', 'stop', 'text']}
                
                    relates = [ i for i in annotations['relate'].values()
                                if  (i.arg1 in entities_from_sentence['entity'].keys()) 
                                and (i.arg2 in entities_from_sentence['entity'].keys())]

                    for rel_i in relates:
                        arg1 = rel_i.arg1
                        ent1 = entities_from_sentence['entity'][arg1]
                        arg2 = rel_i.arg2
                        ent2 = entities_from_sentence['entity'][arg2]

                        tokens = [token.text for token in tokenized_sentence]
                        #e1 = (ner_dict[ent1.name], ent1.start-sentence.start, ent1.stop-sentence.start)
                        #e2 = (ner_dict[ent2.name], ent2.start-sentence.start, ent2.stop-sentence.start)
                        e1 = (ent1.start-sentence.start, ent1.stop-sentence.start)
                        e2 = (ent2.start-sentence.start, ent2.stop-sentence.start)
                        label = rel_2_id[rel_i.name]

                        sample = Sample(sentence.text,
                                        [i.text for i in tokenized_sentence], 
                                        tokens, e1, e2, label)
                        samples.append(sample)
    return samples

## 3.2 train-test split

In [16]:
data = get_re_dataset(glob('train_data/*'))
random.shuffle(data)

train_size = int(len(data)*0.8)
train = data[:train_size]
val = data[train_size:]
test = get_re_dataset(['test_data'])

множество слов и символов

In [17]:
char_set = list( {ch for sample in train for token in sample.tokens for ch in token} )
char_set.insert(0, '<unk>'), char_set.insert(0, '<pad>')
word_set = list( {token for sample in train for token in sample.tokens} )
word_set.insert(0, '<unk>'), word_set.insert(0, '<pad>')

(None, None)

## 3.3 генерация батчей

In [18]:
def get_next_gen_batch(samples, max_seq_len=1024, max_char_seq_len=40, batch_size=1):
    indices = np.arange(len(train))
    np.random.shuffle(indices)
    batch_begin = 0
    #with tqdm(total=len(train)) as pbar:
    while batch_begin < len(train):
          batch_indices = indices[batch_begin: batch_begin + batch_size]
          batch_chars = []
          batch_words = []
          ent_info = []
          batch_labels = []
          batch_max_len = 0
          for data_ind in batch_indices:
              sample = train[data_ind] #беру одно предложение
              words = torch.zeros(max_seq_len, dtype=torch.long).cuda()
              inputs = []
              for token_num, token in enumerate(sample.tokens[:max_seq_len]): #цикл по токенам предложения, обрезанного до max_seq_len
                  #слова
                  words[token_num] = word_set.index(token) if token in word_set else word_set.index('<unk>')
                  #символы
                  chars = [char_set.index(ch) if ch in char_set else char_set.index("<unk>") for ch in token][:max_char_seq_len] #максимальная символьная длина токена - max_char_seq_len
                  chars += [0] * (max_char_seq_len - len(chars)) #каждый токен должен быть представлен max_char_seq_len символами
                  inputs.append(chars)
              batch_max_len = max(batch_max_len, len(inputs)) #кол-во токенов в предложении
              inputs += [[0]*max_char_seq_len] * (max_seq_len - len(inputs)) #приводим предложение к длине max_seq_len
              
              # отношения взятые по символам
              # ent_info.append(((min(sample.e1[0], max_seq_len-1), min(sample.e1[1], max_seq_len-1)), 
              #                 (min(sample.e2[0], max_seq_len-1), min(sample.e2[1], max_seq_len-1)))
              #                 )

              # отношения взятые по словам
              e1 = (sample.e1[0], sample.e1[1])
              e2 = (sample.e2[0], sample.e2[1])
              
              # записываем наблюдение только если оно попало в наблюдаемое предложение (по максимальной длине предложения)
              if (e1[0] < max_seq_len) or (e2[0] < max_seq_len):
                try:
                  # сущность #1
                  e1_txt = [i.text for i in tokenize(sample.text[e1[0]:e1[1]])]
                  e1_start = sample.tokenized_text.index(e1_txt[0])
                  e1_stop = sample.tokenized_text.index(e1_txt[-1])
                  # сущность #2
                  e2_txt = [i.text for i in tokenize(sample.text[e2[0]:e2[1]])]
                  e2_start = sample.tokenized_text.index(e2_txt[0])
                  e2_stop = sample.tokenized_text.index(e2_txt[-1])
                  # запись наблюдений
                  ent_info.append(((e1_start, e1_stop),
                                  (e2_start, e2_stop)))
                  batch_chars.append(inputs)
                  batch_words.append(words)
                  batch_labels.append(sample.label)
                except ValueError:
                    continue
          batch_begin += batch_size
          #pbar.update(batch_size)

          if len(batch_words)>0:
            batch_words = torch.stack(batch_words).cuda()
            batch_chars = torch.cuda.LongTensor(batch_chars)
            labels = torch.cuda.LongTensor(batch_labels)

            yield batch_indices, batch_words, batch_chars, ent_info, labels

In [ ]:
for _, batch_words, batch_chars, batch_ents, labels in get_next_gen_batch(train):
    1+1
    break

print()
print( batch_words.size() )
print( batch_chars.size() )
print( len(batch_ents) )
print( labels.size() )


torch.Size([2, 1024])
torch.Size([2, 1024, 40])
2
torch.Size([2])


## 3.4 модель

In [ ]:
class ReModel(nn.Module):
    def __init__(self, char_set_size, char_embedding_dim=16, classes_count=len(rel_2_id), char_max_seq_len=40, lstm_embedding_dim=16):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.lstm_layer_1 = nn.LSTM(char_max_seq_len * char_embedding_dim, lstm_embedding_dim, batch_first=True, bidirectional=True)
        self.lstm_layer_2 = nn.LSTM(char_max_seq_len * char_embedding_dim, lstm_embedding_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.2)
        self.out_layer = nn.Linear(4*lstm_embedding_dim, classes_count)

    def forward(self, inputs, ents):
        #print(ents[0])

        # обработка токенов
        #print('inputs'.ljust(30), inputs.size())
        projections = self.embeddings_layer.forward(inputs)
        #print('projections'.ljust(30), projections.size())
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        #print('projections'.ljust(30), projections.size())
        _, (final_hidden_state, _) = self.lstm_layer_1(projections)
        rnn_output_1 = torch.cat((final_hidden_state[-2], final_hidden_state[-1]), dim=1)
        rnn_output_1 = self.dropout(rnn_output_1)
        #print('rnn_output_1'.ljust(30), rnn_output_1.size())

        # обработка сущностей
        pr_e1_batch = []
        pr_e2_batch = []
        for (e1, e2) in ents:
          #сущность 1
          pr_e1 = projections[:,e1[0]:e1[1]+1,:]
          pr_e1 = pr_e1.mean(dim=1)
          pr_e1_batch.append(pr_e1)
          #сущность 2
          pr_e2 = projections[:,e2[0]:e2[1]+1,:]
          pr_e2 = pr_e2.mean(dim=1)
          pr_e2_batch.append(pr_e2)
        pr_e1 = torch.stack(pr_e1_batch)
        pr_e2 = torch.stack(pr_e2_batch)
        pr_cat = torch.cat([pr_e1, pr_e2], dim=1)
        #print('pr_cat'.ljust(30), pr_cat.size())
        #lstm над сущностями
        _, (final_hidden_state, _) = self.lstm_layer_2(pr_cat)
        rnn_output_2 = torch.cat((final_hidden_state[-2], final_hidden_state[-1]), dim=1)
        rnn_output_2 = self.dropout(rnn_output_2)
        #print('rnn_output_2'.ljust(30), rnn_output_2.size())
        #объединяем инфо
        com = torch.cat([rnn_output_1, rnn_output_2], dim=1)
        #print('com'.ljust(30), com.size())
        output = self.out_layer.forward(com)
        #print('output'.ljust(30), output.size())
        return output

## 3.5 ф-ция обучения модели

один проход - отладка

In [ ]:
model = ReModel(len(char_set))

lr=0.01
device_name="cuda"
device = torch.device(device_name)

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss().cuda()
model = model.to(device)

logits = model(batch, batch_ents) # Прямой проход
loss = loss_function(logits, labels) # Подсчёт ошибки
loss.backward() # Подсчёт градиентов dL/dw
optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
optimizer.zero_grad() # Занул\ение градиентов, чтобы их спокойно менять на следующей итерации

TypeError: ignored

полный проход

In [ ]:
def train_gen_model(model, train_samples, val_samples, epochs_count=10, 
                    loss_every_nsteps=100, lr=0.01, save_path="model.pt", device_name="cuda",
                    early_stopping=True):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss().cuda()
    prev_avg_val_loss = None
    for epoch in range(epochs_count):
        model.train()
        for step, (_, batch_words, batch_chars, batch_ents, batch_labels) in enumerate(get_next_gen_batch(train)):
            if batch_words.size(0)>0:
              logits = model(batch_words, batch_chars, batch_ents) # Прямой проход
              loss = loss_function(logits, batch_labels) # Подсчёт ошибки
              loss.backward() # Подсчёт градиентов dL/dw
              optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
              optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
              total_loss += loss.item()
        val_total_loss = 0
        val_batch_count = 0
        model.eval()
        for _, (_, batch_words, batch_chars, batch_ents, batch_labels) in enumerate(get_next_gen_batch(val)):
            logits = model(batch_words, batch_chars, batch_ents) # Прямой проход
            val_total_loss += loss_function(logits, batch_labels) # Подсчёт ошибки
            val_batch_count += 1
        avg_val_loss = val_total_loss/val_batch_count
        print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
        total_loss = 0
        start_time = time.time()

        if early_stopping and prev_avg_val_loss is not None and avg_val_loss > prev_avg_val_loss:
            model.load_state_dict(torch.load(save_path))
            model.eval()
            break
        prev_avg_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)

## 3.6 метрики

In [ ]:
def predict(model, samples):
    model.eval()
    tp, fp, fn = 0,0,0
    for _, (_, batch, batch_ents, batch_labels) in enumerate(get_next_gen_batch(samples)):
            logits = model(batch, batch_ents)
            plabels = logits.max(dim=1)[1]
            tp += len([(i,j) for i,j in zip(batch_labels.tolist(), plabels.tolist()) if i==j])
            fp += len([(i,j) for i,j in zip(batch_labels.tolist(), plabels.tolist()) if i!=j])
            fn += len([(i,j) for i,j in zip(batch_labels.tolist(), plabels.tolist()) if i!=j])

    recall, precision = compute_precision_and_recall(tp, fp, fn)
    try:
          f_measure = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
          f_measure = 'None'
    print('precision:', precision)
    #print('recall:   ', recall)
    #print('f1:       ', f_measure)

## --ReModel

In [ ]:
model = ReModel(len(char_set))
train_gen_model(model, train, val, epochs_count=30, early_stopping=False, lr=0.02)

Trainable params: 172011
Epoch = 0, Avg Train Loss = 5.8837, Avg val loss = 1.7912, Time = 60.10s
Epoch = 1, Avg Train Loss = 5.8076, Avg val loss = 1.8443, Time = 60.61s
Epoch = 2, Avg Train Loss = 5.8544, Avg val loss = 1.9233, Time = 62.07s
Epoch = 3, Avg Train Loss = 5.8812, Avg val loss = 1.8286, Time = 62.12s
Epoch = 4, Avg Train Loss = 5.8330, Avg val loss = 1.8156, Time = 62.05s
Epoch = 5, Avg Train Loss = 5.8089, Avg val loss = 1.8610, Time = 61.22s
Epoch = 6, Avg Train Loss = 5.7973, Avg val loss = 1.8575, Time = 61.57s
Epoch = 7, Avg Train Loss = 5.8268, Avg val loss = 1.8676, Time = 62.06s
Epoch = 8, Avg Train Loss = 5.8475, Avg val loss = 1.8522, Time = 61.15s
Epoch = 9, Avg Train Loss = 5.8249, Avg val loss = 1.8259, Time = 61.57s
Epoch = 10, Avg Train Loss = 5.8285, Avg val loss = 1.8529, Time = 60.96s
Epoch = 11, Avg Train Loss = 5.8460, Avg val loss = 1.8263, Time = 61.75s
Epoch = 12, Avg Train Loss = 5.8192, Avg val loss = 1.8396, Time = 61.08s
Epoch = 13, Avg Train L

In [ ]:
print('on train dataset:'.center(30, '-'))
predict(model, train)

print('\n', 'on val dataset:'.center(30, '-'))
predict(model, val)

print('\n', 'on train dataset:'.center(30, '-'))
predict(model, test)

------on train dataset:-------
precision: 0.36665677840403443

 -------on val dataset:--------
precision: 0.3582443653618031

 ------on train dataset:-------
precision: 0.27603143418467585


 # тест

In [ ]:
class ReModel(nn.Module):
    def __init__(self, char_set_size, word_set_size,
                 char_embedding_dim=8, word_embedding_dim=8,
                 classes_count=len(rel_2_id), 
                 char_max_seq_len=40, lstm_embedding_dim=16):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(word_set_size, word_embedding_dim)
        #self.lstm_layer_1 = nn.LSTM(word_embedding_dim, lstm_embedding_dim, batch_first=True, bidirectional=True)
        # self.dropout = nn.Dropout(0.2)
        self.hw = nn.Linear(1024*word_embedding_dim, 2*lstm_embedding_dim)
        self.h1 = nn.Linear(word_embedding_dim, lstm_embedding_dim)
        self.h2 = nn.Linear(word_embedding_dim, lstm_embedding_dim)

        self.out_layer = nn.Linear(4*lstm_embedding_dim, classes_count)
        
    def forward(self, words, chars, ents):
        # обработка токенов
        print('-'*30)
        print(words)
        projections = self.embeddings_layer(words)
        print(projections)
        projections = projections.reshape(projections.size(0),-1)
        # _, (final_hidden_state_1, _) = self.lstm_layer_1(projections)
        # rnn_output_1 = torch.cat((final_hidden_state_1[-2], final_hidden_state_1[-1]), dim=1)
        hw = self.hw(projections)
       
        # обработка сущностей
        pr_e1_batch = []
        pr_e2_batch = []
        for inp_num, (e1, e2) in enumerate(ents):
          #сущность 1
          inp_1 = words[inp_num,e1[0]:e1[1]+1]
          pr_e1 = self.embeddings_layer(inp_1)
          pr_e1 = pr_e1.mean(dim=0)
          pr_e1_batch.append(pr_e1)

          #сущность 2
          inp_2 = words[inp_num,e2[0]:e2[1]+1]
          pr_e2 = self.embeddings_layer(inp_2)
          pr_e2 = pr_e2.mean(dim=0)
          pr_e2_batch.append(pr_e2)
        pr_e1 = torch.stack(pr_e1_batch)
        pr_e2 = torch.stack(pr_e2_batch)
        h1 = self.h1(pr_e1)
        h2 = self.h2(pr_e2)
        h_cat = torch.cat([h1, h1], dim=1)

        #объединяем инфо
        com = torch.cat([hw, h_cat], dim=1)
        output = self.out_layer.forward(com)
        #print(output.size())
        output = torch.log_softmax(output, dim=1)
        return output

In [ ]:
model = ReModel(len(char_set), len(word_set))
train_gen_model(model, train, val, epochs_count=1, early_stopping=False, lr=0.002)

In [ ]:
model.eval()
for step, (_, batch_words, batch_chars, batch_ents, batch_labels) in enumerate(get_next_gen_batch(train)):
            # print(batch)
            logits = model(batch_words, batch_chars, batch_ents)
            plabels = logits.max(dim=1)[1]
            print(logits)
            # print(plabels)
            print()
            # break

------------------------------
tensor([[ 5052, 10943,  6650,  ...,     0,     0,     0]], device='cuda:0')
tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0',
       grad_fn=<EmbeddingBackward0>)
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

------------------------------
tensor([[17134,  2821,   293,  ...,     0,     0,     0]], device='cuda:0')
tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0

KeyboardInterrupt: ignored